## Run a model from Python
### Setup MIKE installation using pythonnet and DHI.Mike.Install

In [ ]:
import clr

clr.AddReference('DHI.Mike.Install, Version=1.0.0.0, Culture=neutral, PublicKeyToken=c513450b5d0bf0bf')

from DHI.Mike.Install import MikeImport
MikeImport.SetupLatest()
print('Found MIKE installation in: ' + MikeImport.ActiveProduct().InstallRoot)

### Load MIKE 1D libaries

In [ ]:
clr.AddReference('DHI.Mike1D.Generic')
clr.AddReference('DHI.Mike1D.Mike1DDataAccess')
clr.AddReference('DHI.Mike1D.SimLauncher')
clr.AddReference('DHI.Mike1D.StructureModule')

from DHI.Mike1D.Generic import Connection, FilePath, Diagnostics
from DHI.Mike1D.Mike1DDataAccess import Mike1DData, Mike1DController, ControllerState
from DHI.Mike1D.SimLauncher import SimulationWorker, SimulationLauncher
from DHI.Mike1D.StructureModule import HonmaWeir

### Run a setup using SimulationWorker

In [ ]:
input_file_name = r'.\exercise1\exercise1.m1dx'
input_file_path = FilePath(input_file_name)
simulationWorker = SimulationWorker(input_file_path)
simulationWorker.Work()
simulationWorker.Dispose()

### Run a setup using Mike1DController

In [ ]:
input_file_name = r'.\exercise1\exercise1.m1dx'
connection = Connection.Create(input_file_name)
controller = Mike1DController.Load(connection)
diagnostics = Diagnostics('My Diagnostics')

controller.Validate()
controller.Initialize(diagnostics)
controller.Prepare()
controller.Run()
controller.Finalize()
controller.Finish()

!{'copy exercise1\Exercise1BaseHD.res1d exercise1\Exercise1BaseHDNoScriptPython.res1d'}

### C# script translated to Python

In [ ]:
class MyScript:

    def __init__(self, controller):
        self.controller = controller
        self.mike1d_data = controller.Mike1DData
        self.structures = controller.Mike1DData.Network.StructureCollection.Structures

        self.crest_width_initial = 0.0
        self.weir_index = -1
        self.weir_id = "Weir"

    def modify_mike1d_data(self):
        """ Method to modify the Mike1DData """
        structures, weir_id = self.structures, self.weir_id
        
        weir_index = list(filter(lambda structure: structure[1].ID == weir_id, enumerate(structures)))[0][0]    
        self.weir_index = weir_index

        bc_weir = structures[weir_index]
        if isinstance(bc_weir, HonmaWeir):
          return

        print("Removing the weir")
        structures.RemoveAt(weir_index)

        print("Creating Honma weir")
        honma_weir = self.create_honma_weir(bc_weir)

        print("Adding Honma weir to network")
        structures.Add(honma_weir)

    def create_honma_weir(self, bc_weir):
        self.crest_width_initial = bc_weir.WeirGeometry.LevelWidths[0].Width

        honma_weir = HonmaWeir()
        
        honma_weir.ID = bc_weir.ID
        honma_weir.Location = bc_weir.Location
        honma_weir.CrestWidth = self.crest_width_initial
        honma_weir.CrestLevel = bc_weir.InvertLevelUpstream
        honma_weir.WeirCoefficient = 1.0
      
        return honma_weir

    def modify_controller(self):
        self.controller.ControllerEvent += self.controller_on_controller_event
     
    def controller_on_controller_event(self, sender, event_args):
        if event_args.State != ControllerState.Running:
            return

        controller = sender

        self.engine_net = controller.EngineNet
        self.engine_net.PreTimeStepEvent += self.pre_time_step_event

    def pre_time_step_event(self, time_n, time_np1, redo_count):
        honma_weir = self.structures[self.weir_index]
        honma_weir.CrestWidth = self.crest_width_initial * (1.0 - time_n.Minute/600.0)
        print('Time:', self.engine_net.EngineTime.TimeNp1.ToString(), " Width:", honma_weir.CrestWidth)


In [ ]:
input_file_name = r'.\exercise1\exercise1.m1dx'
connection = Connection.Create(input_file_name)
controller = Mike1DController.Load(connection)
diagnostics = Diagnostics('My Diagnostics')

my_script = MyScript(controller)
my_script.modify_mike1d_data()
my_script.modify_controller()

controller.Validate()
controller.Initialize(diagnostics)
controller.Prepare()
controller.Run()
controller.Finalize()
controller.Finish()

!{'copy exercise1\Exercise1BaseHD.res1d exercise1\Exercise1BaseHDScriptPython.res1d'}

In [ ]:
import mikeio1d
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('default')

In [ ]:
res1d_no_script = mikeio1d.Res1D('exercise1\Exercise1BaseHDNoScriptPython.res1d')
res1d_script = mikeio1d.Res1D('exercise1\Exercise1BaseHDScriptPython.res1d')

In [ ]:
df_no_script = res1d_no_script.read_all()
df_script = res1d_script.read_all()

In [ ]:
ax = df_no_script['Discharge:SquareChannel:5000'].plot()
ax = df_script['Discharge:SquareChannel:5000'].plot()
ax.set_xlabel('Time')
ax.set_ylabel('Discharge [m$^3$/s]')
plt.show()